In [1]:
import pathlib
import pickle
import random
import re
import sys

import nibabel as nib
import numpy as np
import pandas as pd

# 1. Data preprocessing

In [2]:
# Path to the data folder. This may be different between users
location = '../data/secure/'

data_path = pathlib.Path(location)

### 1.1 Rename folders
We ran into some issues where the extremely deep paths and long filenames were causing errors with our ability to load the data. To correct this issue, we renamed folders by changing the second long folder name to `feature_masks`. For example,

`data/SECURE_KEY/2036311/DPm.1.2.840.113681.2863050713.1318230214.3060.1227/DPm.1.2.840.113681.2863050713.1318230214.3060`

becomes 

`data/SECURE_KEY/2036311/DPm.1.2.840.113681.2863050713.1318230214.3060.1227/feature_masks`

This does not cause any confusion or ambiguity, as both the filenames themselves, as well as the parent folders contain the same information. The directory structure within the directory given above by `location` is then:

```
location
├── 2036311
|   └── DPm.1.2.840.113681.2863050713.1318230214.3060.1227
|       └── feature_masks
|           └── feature
|               └── DPm ... .nii.gz
|               └── ...
|               └── DPm ... .nii.gz
|           └── mask
|               └── DPm ... 0_mask_win_97_sliding_97_mean.nii.gz
|               └── DPm ... _mask.nii.gz
└── ...
```

In [3]:
# Rename folders
for feature_dir in data_path.glob('*/*/*/feature/'):
    parent_dir = feature_dir.parent
    parent_dir.rename(parent_dir.parent / 'feature_masks')

### 1.2 Train/test splitting
We have been given data on roughly 570 patients. There is some ambiguity in the number, though, as we have case/control status on 575 patients, while we have feature maps for 569 patients. Of the 569 patients with features, 114 were cases and the remaining 455 were controls. In the context of breast cancer prediction, this means 114 of the patients eventually developed breast cancer while the others did not.

Among the patients with extracted feature maps, 533 patients had two images (corresponding to left and right breast), while the remaining 36 had an image for one side only. In consultation with Dr. Aimilia Gastounioti, we decided the most sensible approach would be to treat each image as a separate sample. Using this approach, we have 1102 total samples.

We opted for an 80/20 train/test split, a standard split fraction. This means that 455 patients are assigned to the traing set and 114 are assigned to the test set. We took care to ensure that the case/control ratio within both groups reflected the overall distributions. This can be seen below, where 20.2% of the patients assigned to the training set were cases, and 19.3% of the test set were cases. These numbers are not exactly 20%, though they make sense in light of the fact that the number of patients is not evenly divisible in the fraction we desire.

To eliminate a source of bias in our model, we did not allow patients with multiple images to have their data split between training and testing data. This means that patients with two images always had both images together, and we split the data by patients rather than by sample.

In [4]:
# Train/test split
# Get list of patients with feature maps
patients_list = [subdir.name for subdir in data_path.glob('*/') if subdir.is_dir()]

# Read in case/control information
case_control_df = pd.read_excel('../controlcase.xlsx')

# Create a dictionary mapping patient_id to case/control status
patient_id_to_case = case_control_df[['DummyID', 'Class']].set_index('DummyID')['Class'].to_dict()

# Set random seed so that split can be done reproducibly
np.random.seed(0)

# Pick patients whose images will be in train/test sets
training_patients = np.random.choice(patients_list, replace=False, size=455)
testing_patients = [patient for patient in patients_list if patient not in training_patients]

In [5]:
# Verify the train/test split sizes
print(f'Training patients: {len(training_patients)}\n'
      f'Testing patients: {len(testing_patients)}\n')

# Verify the relative numbers of cases and controls between training and testing
num_training_cases = sum([patient_id_to_case[int(patient_id)] for patient_id in training_patients])
num_testing_cases = sum([patient_id_to_case[int(patient_id)] for patient_id in testing_patients])

print(f'Percent cases in training data: {num_training_cases / len(training_patients)}\n'
      f'Percent cases in testing data: {num_testing_cases / len(testing_patients)}')

Training patients: 455
Testing patients: 114

Percent cases in training data: 0.2021978021978022
Percent cases in testing data: 0.19298245614035087


#### 1.2.1 Replicate training cases to class balance training data

In [6]:
# Replicate the number of training cases
is_train_case = [patient_id_to_case[int(patient_id)] for patient_id in training_patients]
training_patients = np.concatenate((
    3 * [case for i, case in enumerate(training_patients) if is_train_case[i]],
    training_patients
))

In [7]:
# Verify the train/test split sizes
print(f'Training patients: {len(training_patients)}\n'
      f'Testing patients: {len(testing_patients)}\n')

# Verify the relative numbers of cases and controls between training and testing
num_training_cases = sum([patient_id_to_case[int(patient_id)] for patient_id in training_patients])
num_testing_cases = sum([patient_id_to_case[int(patient_id)] for patient_id in testing_patients])

print(f'Percent cases in training data: {num_training_cases / len(training_patients)}\n'
      f'Percent cases in testing data: {num_testing_cases / len(testing_patients)}')

Training patients: 731
Testing patients: 114

Percent cases in training data: 0.5034199726402189
Percent cases in testing data: 0.19298245614035087


### 1.3 Load and process feature maps
Below, we extract all feature maps, apply the mask, sort, and combine features into 4D arrays. Then, we normalize features first across samples then within samples, just as was performed in the code provided for us.

Throughout the process, we are very careful to ensure that features are always in correspondence with their patient_id or case/control status. 

#### 1.3.1 Load data into lists of feature dictionaries

In [8]:
def patient_id_list_to_features(patient_list):
    features = list()
    classes = list()
    
    for patient_id in patient_list:
        # Get patient's case/control status
        patient_class = patient_id_to_case[int(patient_id)]
        
        # Iterate over potentially two samples
        sample_paths = data_path.glob(f'{patient_id}/*')
        for sample in sample_paths:
            mask_path = next(sample.glob('feature_masks/mask/*_mean.nii.gz')).as_posix()
            mask = nib.load(mask_path).get_data().T
            
            patient_features = dict()
            features_paths = sample.glob('feature_masks/feature/*.nii.gz')
            for feature_path in features_paths:

                # Load feature map and apply mask
                feature_map = np.nan_to_num(nib.load(feature_path.as_posix()).get_data().T)
                masked_feature_map = np.multiply(feature_map, mask)

                # Extract the feature name from its filename. Eg: norm_win_97_sliding_97_box_counting from
                # DPm.1.2.840.113681.2863050709.1375427076.3328_norm_win_97_sliding_97_box_counting.nii.gz
                feature_name = re.search('(?<=_).+(?=\.nii\.gz)', feature_path.name).group()  # noqa: W605
                patient_features[feature_name] = masked_feature_map

            features.append(patient_features)
            classes.append(patient_class)
    return (features, classes)
        

random.shuffle(training_patients)
train_features, train_classes = patient_id_list_to_features(training_patients)
test_features, test_classes = patient_id_list_to_features(testing_patients)

In [9]:
len(train_classes), len(test_classes)

(1414, 222)

In [10]:
sum(train_classes) / len(train_classes)

0.5035360678925035

In [11]:
sum(test_classes) / len(test_classes)

0.1891891891891892

#### 1.3.2 Combine the data into 4D arrays
Very importantly, ensure that the features are always ordered the same way for every sample.

In [12]:
# Save the data in 4D arrays

# Create an ordered list of feature names to ensure they are in the same
# order for every sample in the training and testing data
ordered_feature_names = sorted(train_features[0].keys())

# Save the data in 4D arrays
train_data = np.zeros((len(train_features), 34, 26, 29))
test_data = np.zeros((len(test_features), 34, 26, 29))

for sample_number, sample_dict in enumerate(train_features):
    for feature_number, feature_name in enumerate(ordered_feature_names):
        # Crop images to all be 34 x 26. Some are originally larger at 42 x 37
        train_data[sample_number, :, :, feature_number] = sample_dict[feature_name][0:34, 0:26]

for sample_number, sample_dict in enumerate(test_features):
    for feature_number, feature_name in enumerate(ordered_feature_names):
        # Crop images to all be 34 x 26. Some are originally larger at 42 x 37
        test_data[sample_number, :, :, feature_number] = sample_dict[feature_name][0:34, 0:26]

# Convert label lists to numpy arrays
train_classes = np.asarray(train_classes)
test_classes = np.asarray(test_classes)

#### 1.3.3 Normalize the feature maps
As was done in the preprocessing code from the 2016 paper, we first normalize across samples, then normalize features within samples. Note that we add a term, `epsilon` to the divisors below. This is because some features are zero across all samples or across all feature_maps within sample. In these cases, we would be dividing by zero, which would introduce unwanted `nan` terms into the data.

In [13]:
epsilon = 1e-8

# Normalize the data across samples
# Combine the data and find the largest magnitude values for each feature
full_data = np.concatenate((train_data, test_data))
max_image = np.abs(full_data).max(axis=0)

train_data = np.divide(train_data, max_image + epsilon)
test_data = np.divide(test_data, max_image + epsilon)

# Normalize feature maps within samples so that the maximum value in each is 1.
# # This is the within-sample normalization that was performed
# # in the preprocessing code we received from the 2016 paper
for data_source in (train_data, test_data):
    for sample_number, sample in enumerate(data_source):
        for feature_number in range(29):
            feature_map = sample[:, :, feature_number]
            max_val = np.abs(feature_map).max()
            data_source[sample_number, :, :, feature_number] = np.divide(feature_map, max_val + epsilon)

# Save the data as pickled tuples of data, labels
training_set = (train_data, train_classes)
testing_set = (test_data, test_classes)

train_data_path = data_path.parent.joinpath('train_data.pkl')
test_data_path = data_path.parent.joinpath('test_data.pkl')

with open(train_data_path, 'wb') as f:
    pickle.dump(training_set, f)

with open(test_data_path, 'wb') as f:
    pickle.dump(testing_set, f)

# 2. Create and train CNN model

In [14]:
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Conv2D, Activation, Flatten, MaxPooling2D, Dropout, SpatialDropout2D
from keras.models import Sequential
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.metrics import roc_auc_score
from tensorflow import set_random_seed

Using TensorFlow backend.


In [15]:
train_classes = to_categorical(train_classes)
test_classes = to_categorical(test_classes)

In [23]:
# Set numpy and TensorFlow random seeds in the hopes of making
# results reproducible. This will not be possible when using a GPU,
# as there may be asynchronous processing for which no random seed
# could account.
set_random_seed(2)
np.random.seed(1)

datagen = ImageDataGenerator()
datagen.fit(train_data)

val_datagen = ImageDataGenerator()
val_datagen.fit(test_data)

model = Sequential([
    Conv2D(10, kernel_size=(5, 5), activation='tanh',
           data_format='channels_last', input_shape=(34, 26, 29)),
    MaxPooling2D(pool_size=(2, 2)),
#     Dropout(0.2),
    Conv2D(10, kernel_size=(4, 3), activation='tanh'),
    MaxPooling2D(pool_size=(2, 2)),
#     Dropout(0.4),
    Flatten(),
    Dense(5, activation='tanh'),
    Dense(2, activation='sigmoid')
])

sgd = SGD(lr=0.01)
model.compile(optimizer=sgd, loss='binary_crossentropy',
              metrics=['binary_accuracy'])

callback = EarlyStopping(monitor='val_loss', min_delta=-0.1, patience=3,
                         verbose=1, mode='auto', baseline=0.8)

# class_weights = {0: 1, 1: 4}
model.fit_generator(datagen.flow(train_data, train_classes, batch_size=1, shuffle=True),
#                     callbacks=[callback],
                    steps_per_epoch=len(train_data), epochs=50,
#                     class_weight=class_weights,
                    validation_data=val_datagen.flow(test_data, test_classes),
                    nb_val_samples=test_data.shape[0])

score = model.evaluate(test_data, test_classes)

print("Weighted test accuracy: ", score[1])
preds = model.predict(test_data)
auc = roc_auc_score(test_classes, preds)
print(model.summary())
print(f"AUROC: {auc}")

/home/michael/miniconda3/envs/cis537/lib/python3.6/site-packages/ipykernel/__main__.py:40: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/home/michael/miniconda3/envs/cis537/lib/python3.6/site-packages/ipykernel/__main__.py:40: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=1414, epochs=50, validation_data=<keras_pre..., validation_steps=222)`


Epoch 1/50
1414/1414 [==============================] - 3s 2ms/step - loss: 0.6957 - binary_accuracy: 0.5325 - val_loss: 0.6148 - val_binary_accuracy: 0.7407
Epoch 2/50
1414/1414 [==============================] - 3s 2ms/step - loss: 0.6750 - binary_accuracy: 0.5792 - val_loss: 0.5810 - val_binary_accuracy: 0.7322
Epoch 3/50
1414/1414 [==============================] - 3s 2ms/step - loss: 0.6418 - binary_accuracy: 0.6368 - val_loss: 0.6310 - val_binary_accuracy: 0.6457
Epoch 4/50
1414/1414 [==============================] - 3s 2ms/step - loss: 0.5867 - binary_accuracy: 0.7111 - val_loss: 0.7157 - val_binary_accuracy: 0.5615
Epoch 5/50
1414/1414 [==============================] - 3s 2ms/step - loss: 0.5273 - binary_accuracy: 0.7348 - val_loss: 0.6876 - val_binary_accuracy: 0.6080
Epoch 6/50
1414/1414 [==============================] - 3s 2ms/step - loss: 0.4605 - binary_accuracy: 0.7769 - val_loss: 0.6070 - val_binary_accuracy: 0.7127
Epoch 7/50
1414/1414 [==============================

In [24]:
# Check AUC on the training data, just to verify that the training data was learned.
score = model.evaluate(train_data, train_classes)

print("Weighted test accuracy: ", score[1])
preds = model.predict(train_data)
auc = roc_auc_score(train_classes, preds)
print(model.summary())
print(f"AUROC: {auc}")

1414/1414 [==============================] - 0s 104us/step
Weighted test accuracy:  0.937057991513437
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 30, 22, 10)        7260      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 11, 10)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 12, 9, 10)         1210      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 6, 4, 10)          0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 240)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 1205      
________________________________________

In [25]:
model.save('../model/most_recent.h5')